In [ ]:
import cv2 
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 
import imutils
import copy
from scipy.ndimage.filters import  gaussian_filter
import math
import skimage.transform as sktr


In [ ]:
raisi = cv2.imread("./inputs/P5/raisi.png",0)
rouhani = cv2.imread("./inputs/P5/rouhani.png",0)
biden = cv2.imread("./inputs/P5/biden.png",0)
trump = cv2.imread("./inputs/P5/trump.png",0)

### Fourier

In [ ]:
def get_fourier(img):
    img = img.copy()
    img_dft = cv2.dft(np.float32(img),flags=cv2.DFT_COMPLEX_OUTPUT)
    dft_shift = np.fft.fftshift(img_dft)
    magnitude, angle = cv2.cartToPolar(dft_shift[:, :, 0],dft_shift[:, :, 1])
    magnitude = np.log(magnitude)
    return magnitude, angle
    

### Align / by: https://github.com/daggerdagr/cs194-26-proj3/blob/master/align_image_code.py

In [ ]:
def get_points(t):
    # extracted manually
    if t =="trump-biden":
        return ((427.6385281385282, 229.80303030303025), (421.1450216450218, 439.7597402597402), (412.4870129870131, 247.1190476190476), (403.8290043290045, 478.7207792207792))
    else:
        return ((343.7424242424243, 246.08008658008652), (350.6688311688312, 334.39177489177484), (312.57359307359314, 249.54329004329003), (322.96320346320346, 330.9285714285714))

def recenter(im, r, c):
    R, C = im.shape
    rpad = (int) (np.abs(2*r+1 - R))
    cpad = (int) (np.abs(2*c+1 - C))
    return np.pad(im, [(0 if r > (R-1)/2 else rpad, 0 if r < (R-1)/2 else rpad),
             (0 if c > (C-1)/2 else cpad, 0 if c < (C-1)/2 else cpad)], 'constant')

def find_centers(p1, p2):
    cx = np.round(np.mean([p1[0], p2[0]]))
    cy = np.round(np.mean([p1[1], p2[1]]))
    return cx, cy

def align_image_centers(im1, im2, pts):
    p1, p2, p3, p4 = pts
    
    cx1, cy1 = find_centers(p1, p2)
    cx2, cy2 = find_centers(p3, p4)

    im1 = recenter(im1, cy1, cx1)
    im2 = recenter(im2, cy2, cx2)
    return im1, im2

def rescale_images(im1, im2, pts):
    p1, p2, p3, p4 = pts
    len1 = np.sqrt((p2[1] - p1[1])**2 + (p2[0] - p1[0])**2)
    len2 = np.sqrt((p4[1] - p3[1])**2 + (p4[0] - p3[0])**2)
    dscale = len2/len1
    if dscale < 1:
        im1 = sktr.rescale(im1, dscale)
    else:
        im2 = sktr.rescale(im2, 1./dscale)
    return im1, im2

def rotate_im1(im1, im2, pts):
    p1, p2, p3, p4 = pts
    theta1 = math.atan2(-(p2[1] - p1[1]), (p2[0] - p1[0]))
    theta2 = math.atan2(-(p4[1] - p3[1]), (p4[0] - p3[0]))
    dtheta = theta2 - theta1
    im1 = sktr.rotate(im1, dtheta*180/np.pi)
    return im1, dtheta

def match_img_size(im1, im2):
    # Make images the same size
    h1, w1 = im1.shape
    h2, w2= im2.shape
    if h1 < h2:
        im2 = im2[int(np.floor((h2-h1)/2.)) : int(-np.ceil((h2-h1)/2.)), :]
    elif h1 > h2:
        im1 = im1[int(np.floor((h1-h2)/2.)) : int(-np.ceil((h1-h2)/2.)), :]
    if w1 < w2:
        im2 = im2[:, int(np.floor((w2-w1)/2.)) : int(-np.ceil((w2-w1)/2.))]
    elif w1 > w2:
        im1 = im1[:, int(np.floor((w1-w2)/2.)) : int(-np.ceil((w1-w2)/2.))]
    assert im1.shape == im2.shape
    return im1, im2

def align_images(im1, im2,t):
    pts = get_points(t)
    return align_images_w_pts(im1, im2, pts)

def align_images_w_pts(im1, im2, pts):
    im1, im2 = align_image_centers(im1, im2, pts)
    im1, im2 = rescale_images(im1, im2, pts)
    im1, angle = rotate_im1(im1, im2, pts)
    im1, im2 = match_img_size(im1, im2)
    return im1, im2


### A

In [ ]:
def report_align(img1,img2,t1,t2,flag="trump-biden"):
    plt.rcParams['figure.figsize'] = [9, 6]
    new_img1,new_img2 = align_images(img1,img2,flag)
    plt.subplot(2,3,1)
    plt.imshow(img1,cmap="gray")
    plt.title("Source of "+t1)
    plt.axis("off")
    
    plt.subplot(2,3,2)
    plt.imshow(new_img1,cmap="gray")
    plt.title("Align of "+t1)
    plt.axis("off")
    
    m,_ = get_fourier(img1)
    plt.subplot(2,3,3)
    plt.imshow(m,cmap="gray")
    plt.title("Amplitude of aligned "+t1)
    plt.axis("off")
    
    plt.subplot(2,3,4)
    plt.imshow(img2,cmap="gray")
    plt.title("Source of "+t2)
    plt.axis("off")
    
    plt.subplot(2,3,5)
    plt.imshow(new_img2,cmap="gray")
    plt.title("Align of "+t2)
    plt.axis("off")
    
    m,r = get_fourier(new_img2)
    plt.subplot(2,3,6)
    plt.imshow(m,cmap="gray")
    plt.title("Amplitude of aligned "+t2)
    plt.axis("off")
    plt.tight_layout()
    plt.show()
    
    return new_img1,new_img2
    
    
raisi_n,rouhani_n = report_align(raisi,rouhani,"Raisi","Rouhani","raisi-rouhani")
rouhani_n = rouhani_n/255.

biden_n,trump_n = report_align(biden,trump,"Biden","Trump","trump-biden")

### B

In [ ]:
def report_filter(img1,img2,t1="",t2="",coe=4,show=True):
    plt.rcParams['figure.figsize'] = [12, 6]

    plt.subplot(2,3,1)
    plt.imshow(img1,cmap="gray")
    plt.axis("off")
    plt.title("Image of {0} To filter Gaussian blur".format(t1))
    
    
    img1_b = gaussian_filter(img1,coe)
    plt.subplot(2,3,2)
    plt.imshow(img1_b,cmap="gray")
    plt.axis("off")
    plt.title("Image of {0} after filter Gaussian blur".format(t1))
    
    
    m,_ = get_fourier(img1_b)
    plt.subplot(2,3,3)
    plt.imshow(m,cmap="gray")
    plt.title("Amplitude of {0} filtered".format(t1))
    plt.axis("off")
    
    
    
    img2_b = gaussian_filter(img2,coe)
    img2_high = cv2.subtract(img2.copy(),img2_b)
    plt.subplot(2,3,4)
    plt.imshow(img2,cmap="gray")
    plt.axis("off")
    plt.title("Image of {0} To Highpass filter".format(t2))
    
    plt.subplot(2,3,5)
    plt.imshow(img2_high,cmap="gray")
    plt.axis("off")
    plt.title("Image of {0} after Highpass filter".format(t2))
    
    m,_ = get_fourier(img2_high)
    plt.subplot(2,3,6)
    plt.imshow(m,cmap="gray")
    plt.title("Amplitude of {0} filtered".format(t2))
    plt.axis("off")
   
   
    if show:
        plt.tight_layout()
        plt.show()
    else:
        plt.clf()
    
    return img1_b,img2_high
   

_,__ = report_filter(rouhani_n.copy(),raisi_n.copy(),"Rouhani","Raisi")
_,__ = report_filter(biden_n.copy(),trump_n.copy(),"Trump","Biden")

### C

In [ ]:
def report_merge(img1,img2):
    res = np.zeros_like(img1)
    res += img2 
    res += img1
    res = res[65:,:]
    m,_ = get_fourier(res)
    
    return res,m
    
    
biden_f , trump_f = report_filter(biden_n,trump_n,coe=5,show=False)
res1,f1 = report_merge(biden_f,trump_f)

rouhani_f , raisi_f = report_filter(rouhani_n,raisi_n,coe=5,show=False)
res2,f2 = report_merge(raisi_f,rouhani_f)
plt.rcParams['figure.figsize'] = [12, 6]

plt.subplot(2,2,1)
plt.imshow(res1,cmap="gray")
plt.axis("off")
plt.title("Biden-Trump merge")

plt.subplot(2,2,2)
plt.imshow(res2,cmap="gray")
plt.axis("off")
plt.title("Rouhani-Raisi merge")

plt.subplot(2,2,3)
plt.imshow(f1,cmap="gray")
plt.axis("off")
plt.title("Biden-Trump merge Amplitude")

plt.subplot(2,2,4)
plt.imshow(f2,cmap="gray")
plt.axis("off")
plt.title("Rouhani-Raisi Amplitude")

plt.tight_layout()
plt.show() 

### D

In [ ]:
_list = []
for i in np.arange(1,22,4):
    biden_f , trump_f = report_filter(biden_n,trump_n,coe=i,show=False)
    res1,f1 = report_merge(biden_f,trump_f)
    _list.append(res1)
    
plt.rcParams['figure.figsize'] = [24, 9]
h1 = cv2.hconcat(_list[:3])
h2 = cv2.hconcat(_list[3:])
plt.imshow(cv2.vconcat([h1,h2]),cmap="gray")
plt.axis("off")
plt.title("Biden-Trump Transformation")
plt.show()


_list = []
for i in np.arange(1,22,4):
    rouhani_f , raisi_f = report_filter(rouhani_n,raisi_n,coe=i,show=False)
    res2,f2 = report_merge(raisi_f,rouhani_f)
    _list.append(res2)
    
plt.rcParams['figure.figsize'] = [24, 9]
h1 = cv2.hconcat(_list[:3])
h2 = cv2.hconcat(_list[3:])
plt.imshow(cv2.vconcat([h1,h2]),cmap="gray")
plt.axis("off")
plt.title("Rouhani-Raisi Transformation")
plt.show()

### Make Video

In [ ]:
_list_1 = []
for i in np.arange(0.5,15,0.2):
    biden_f , trump_f = report_filter(biden_n,trump_n,coe=i,show=False)
    res1,f1 = report_merge(biden_f,trump_f)
    _list_1.append(res1)

h,w = _list_1[0].shape
out = cv2.VideoWriter('./outputs/P5/Output-Biden-Trump Transformation.mp4',cv2.VideoWriter_fourcc(*'MP4V'), 8, (w,h))
for i in range(len(_list_1)):
    img = _list_1[i]
    img = 255 * img
    img = img.astype(np.uint8)
    img = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
    out.write(img)
out.release()


_list_2 = []
for i in np.arange(0.5,15,0.2):
    rouhani_f , raisi_f = report_filter(rouhani_n,raisi_n,coe=i,show=False)
    res2,f2 = report_merge(raisi_f,rouhani_f)
    _list_2.append(res2)
    

h,w = _list_2[0].shape
out = cv2.VideoWriter('./outputs/P5/Output-Rouhani-Raisi Transformation.mp4',cv2.VideoWriter_fourcc(*'MP4V'), 8, (w,h))
for i in range(len(_list_2)):
    img = _list_2[i]
    img = 255 * img
    img = img.astype(np.uint8)
    img = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
    out.write(img)
out.release()